In [192]:
from Dataload import dataload

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils

from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np

from torchvision import transforms

import pandas as pd
from module import densenet

from tqdm.notebook import tqdm

In [459]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")


    

In [460]:
class multi_frames():
    def __init__(self,frame_list,channel=3,img_width=256,img_length=256,embed_dim=1000,frame_embedding=None):
        self.frame_list=frame_list
        
        if frame_embedding is None: 
            self.frames = torch.empty(size=(len(frame_list), channel, img_width,img_length)) 
            for index,element in enumerate(frame_list):
                self.frames[index]=element.frame_img
        else:
            self.frames=frame_embedding
    def __len__(self):
        return len(self.frame_list)

        

class frame():
    def __init__(self,frame_img,frame_name,max_face=5):
        self.frame_img=frame_img
        self._file_name=frame_name
        self.transform_face=transforms.Compose([
                                 transforms.Resize((32,32)), 
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
        
        self.transform_frame=transforms.Compose([
                     transforms.Resize((256,256)),
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])

        
        def face_extraction(img,max_face):
            bounding_boxes, landmarks = detect_faces(img)
            img_list=[]
            for box_index,(left,right,up,bottom,_) in enumerate(bounding_boxes):
                cropped_img=img.crop((left,right,up,bottom))
                img_list.append(self.transform_face(cropped_img))
                if len(img_list)==max_face:
                    break
            while len(img_list) !=max_face:
                END_PAD= Image.new(mode = "RGB", size = (256, 256), color =(0, 0, 0))
                img_list.append(self.transform_face(END_PAD))
            img_list=torch.stack(img_list)

            return img_list
        self.faces=face_extraction(self.frame_img,max_face)


        
        self.frame_img=self.transform_frame(frame_img)

        
        
    
        
        

In [464]:
import os
from constant import EMOTIPATH


Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")


train_data=Video_Frame_Data(Train_label,"train.csv",base_path_v=Train_video,frame_num=5,embedding=False)
#valid_data=Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=5)

In [465]:
test=train_data[0][0]

[ 0  5 11 17 23]


In [466]:
test.frames.shape

torch.Size([5, 1000])

In [478]:
class Test(nn.Module):
    
    def __init__(self):
        super(Test, self).__init__()

        self.densenet=densenet.densenet121(pretrained=True)
        self.dim_face=100
        self.dim_frame=1000
        
        self.fc1=nn.Linear(1000,1)

    def check_raw(self,x,dim):
        if x.shape[-1]==dim
            return True
        return False
    
    def forward(self,x):
        print(self.check_embedding(x))
        
        return self.fc1(x)
        

In [479]:
model=Test()

In [483]:
model(torch.rand(1,1000))

False


tensor([[-0.9991]], grad_fn=<AddmmBackward>)

In [484]:
x=torch.rand(4,2,3)

In [486]:
x.shape[-1]

3